In [1]:
### vergence

## dependnecies
import data_preparation as dp
import pandas as pd
import numpy as np
import os

In [2]:
# df_all = pd.read_csv(
#         r"C:\Users\Julia\Desktop\Daten-BA\SMI_RawData\Raw Data - Raw Data - 01.txt",
#         na_values = "-"
#     )

C:\Users\Julia\AppData\Local\Temp\ipykernel_31216\3516985476.py:1: DtypeWarning: Columns (46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(


In [2]:
# Define paths
read_path =  r"W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\02_BeGaze\SMI_RawData" # r"C:\Users\Julia\Desktop\Daten-BA\SMI_RawData"
save_path = r"W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\04_Prepared_data\00_Julia"
excluded_path = r'W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\02_Excluded_ids.csv'

In [3]:
def read_data(filename: str) -> pd.DataFrame:
    """
    input: 
        filename as "Raw Data - Raw Data - 01.txt"
    output: 
        dataframe for one file, which is one participants
    """
        
    df_all = pd.read_csv(
        read_path + "\\" + filename, 
        na_values = "-"
    )
#    df_all = pd.read_csv(
#        r"W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\02_BeGaze\SMI_RawData" + "\\" + filename, 
#        na_values = "-"
    
    return df_all
    

In [4]:
#df_all.columns

Index(['RecordingTime [ms]', 'Time of Day [h:m:s:ms]', 'Trial', 'Stimulus',
       'Export Start Trial Time [ms]', 'Export End Trial Time [ms]',
       'Participant', 'Color', 'Tracking Ratio [%]', 'Category Group',
       'Category Right', 'Category Left', 'Index Right', 'Index Left',
       'Pupil Size Right X [px]', 'Pupil Size Right Y [px]',
       'Pupil Diameter Right [mm]', 'Pupil Size Left X [px]',
       'Pupil Size Left Y [px]', 'Pupil Diameter Left [mm]',
       'Point of Regard Right X [px]', 'Point of Regard Right Y [px]',
       'Point of Regard Left X [px]', 'Point of Regard Left Y [px]',
       'Gaze Vector Right X', 'Gaze Vector Right Y', 'Gaze Vector Right Z',
       'Gaze Vector Left X', 'Gaze Vector Left Y', 'Gaze Vector Left Z',
       'Eye Position Right X [mm]', 'Eye Position Right Y [mm]',
       'Eye Position Right Z [mm]', 'Eye Position Left X [mm]',
       'Eye Position Left Y [mm]', 'Eye Position Left Z [mm]',
       'Pupil Position Right X [px]', 'Pupil Pos

In [4]:
# get 10 last seconds of one video
### cut last 10 seconds of videos ###

def cut_last_10sec_of_one_video(df: pd.DataFrame) -> pd.DataFrame:
    """
    returns the data of the last 10s of a video (starting by the last event, which is in the video)

    input:
        data (sorted by time) for one video of one participant
    output:
        data of the last 10sec of the input data
    """

    # number of last seconds
    video_end_time_ms = df.loc[:, "RecordingTime [ms]"].max()
    video_10sec_before_end_time_ms = video_end_time_ms - 10_000 # -10_000ms

    #get df last seconds: 
    #df_vid_final = df.loc[(df_event_start_trial > video_10sec_before_end_time_ms) & (df_event_end_trial <= video_end_time_ms), :]
    df_vid_final = df.loc[(df["RecordingTime [ms]"] > video_10sec_before_end_time_ms) & (df["RecordingTime [ms]"] <= video_end_time_ms), :]

    return df_vid_final


def cut_last_10sec_of_videos(df: pd.DataFrame) -> pd.DataFrame:
    """
    returns the data of the last 10s of all videos (starting by the last event, which is in the video)

    input: 
        data (sorted by time) for of one participant
    output: 
        data of the last 10sec of the input data

    """
    df_split = df.groupby("Stimulus")
    ## necessary data together
    df_concat = pd.DataFrame()
    for i in range(1,16):
        # hier existiert einmal ein video nicht, vp 13
        # if abfrage ob das vid existiert
        if "vid" + str(i) in df_split.groups.keys():
            df_temp = pd.DataFrame(df_split.get_group("vid" + str(i)))
            df_concat = pd.concat([df_concat, cut_last_10sec_of_one_video(df_temp)]) 
    return df_concat


In [30]:
df = cut_last_10sec_of_videos(df_all)
df["Stimulus"].unique()
df

,RecordingTime [ms],Time of Day [h:m:s:ms],Trial,Stimulus,Export Start Trial Time [ms],Export End Trial Time [ms],Participant,Color,Tracking Ratio [%],Category Group,...,Pupil Position Left Y [px],Port Status,Annotation Name,Annotation Description,Annotation Tags,Mouse Position X [px],Mouse Position Y [px],Scroll Direction X,Scroll Direction Y,Content
237762,1095835.478,10:02:49:472,Trial043,vid1,0.0,201054.568,1,CornflowerBlue,85.268,Eye,...,643.905,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237763,1095839.482,10:02:49:476,Trial043,vid1,0.0,201054.568,1,CornflowerBlue,85.268,Eye,...,643.870,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237764,1095843.487,10:02:49:480,Trial043,vid1,0.0,201054.568,1,CornflowerBlue,85.268,Eye,...,643.930,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237765,1095847.491,10:02:49:484,Trial043,vid1,0.0,201054.568,1,CornflowerBlue,85.268,Eye,...,644.009,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237766,1095851.495,10:02:49:488,Trial043,vid1,0.0,201054.568,1,CornflowerBlue,85.268,Eye,...,644.006,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056591,4432514.359,10:58:26:151,Trial085,vid15,0.0,214521.271,1,CornflowerBlue,80.971,Eye,...,0.000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1056592,4432518.364,10:58:26:155,Trial085,vid15,0.0,214521.271,1,CornflowerBlue,80.971,Eye,...,0.000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1056593,4432522.364,10:58:26:159,Trial085,vid15,0.0,214521.271,1,CornflowerBlue,80.971,Eye,...,0.000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1056594,4432526.365,10:58:26:163,Trial085,vid15,0.0,214521.271,1,CornflowerBlue,80.971,Eye,...,0.000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# get gaze vectors
df_gaze_vectors = df.loc[:, ["Stimulus", "Gaze Vector Right X", "Gaze Vector Right Y", "Gaze Vector Right Z",
       "Gaze Vector Left X", "Gaze Vector Left Y", "Gaze Vector Left Z"]]
df_gaze_vectors

,Stimulus,Gaze Vector Right X,Gaze Vector Right Y,Gaze Vector Right Z,Gaze Vector Left X,Gaze Vector Left Y,Gaze Vector Left Z
237762,vid1,-0.117,0.203,-0.972,-0.136,0.215,-0.967
237763,vid1,-0.124,0.202,-0.971,-0.138,0.216,-0.967
237764,vid1,-0.128,0.201,-0.971,-0.138,0.212,-0.967
237765,vid1,-0.119,0.198,-0.973,-0.136,0.208,-0.969
237766,vid1,-0.113,0.196,-0.974,-0.139,0.208,-0.968
...,...,...,...,...,...,...,...
1056591,vid15,0.000,0.000,0.000,0.000,0.000,0.000
1056592,vid15,0.000,0.000,0.000,0.000,0.000,0.000
1056593,vid15,0.000,0.000,0.000,0.000,0.000,0.000
1056594,vid15,0.000,0.000,0.000,0.000,0.000,0.000


In [32]:
gaze_vektor_right = df_gaze_vectors.loc[:, ["Gaze Vector Right X", "Gaze Vector Right Y","Gaze Vector Right Z"]].values
gaze_vektor_left = df_gaze_vectors.loc[:, ["Gaze Vector Left X", "Gaze Vector Left Y","Gaze Vector Left Z"]].values

In [5]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)) #arcos um winkel zu bekommen

In [46]:
gaze_vektor_left = df.loc[:, ["Gaze Vector Left X", "Gaze Vector Left Y","Gaze Vector Left Z"]].values
gaze_vektor_right = df.loc[:, ["Gaze Vector Right X", "Gaze Vector Right Y","Gaze Vector Right Z"]].values
result = []
for vector in range(len(gaze_vektor_left)):
    vec1 = gaze_vektor_right[vector]
    vec2 = gaze_vektor_left[vector]
    result.append(angle_between(unit_vector(vec1), unit_vector(vec2)))

C:\Users\Julia\AppData\Local\Temp\ipykernel_31216\1287561581.py:3: RuntimeWarning: invalid value encountered in divide
  return vector / np.linalg.norm(vector)


In [6]:
# veregnce angles
# 10 seconds of videos
# gaze vectors
# calcualte angle between gaze vector left and right

def vergence_angles(df):
    gaze_vektor_left = df.loc[:, ["Gaze Vector Left X", "Gaze Vector Left Y","Gaze Vector Left Z"]].values
    gaze_vektor_right = df.loc[:, ["Gaze Vector Right X", "Gaze Vector Right Y","Gaze Vector Right Z"]].values
    result = []
    for vector in range(len(gaze_vektor_left)):
        vec1 = gaze_vektor_right[vector]
        vec2 = gaze_vektor_left[vector]
        result.append(angle_between(unit_vector(vec1), unit_vector(vec2)))
    return result

In [7]:
# pupil distance
# abstand zwischen linker und rechter fixation
# fixationspunkt der Augen

#'Pupil Position Right X [px]', 'Pupil Position Right Y [px]',
#      'Pupil Position Left X [px]', 'Pupil Position Left Y [px]',
def vergence_distance(df):
    x_right = df["Pupil Position Right X [px]"]
    y_right = df["Pupil Position Right Y [px]"]

    x_left = df["Pupil Position Left X [px]"]
    y_left = df["Pupil Position Left Y [px]"]

    distance = np.sqrt((x_right-x_left)**2 + (y_right-y_left)**2)
    return distance

In [37]:
a = vergence_distance(df)
df["vergence distance [px]"] = a
groups = df.groupby(["Stimulus"])
groups.agg({"vergence distance [px]": ["mean", "std"]})
#df.aggregate[{"vergence distance [px]": ["mean", "std"]}]

vergence distance [px]            
                           mean         std
Stimulus                                   
vid1                 219.578560  126.804463
vid10                220.563184  106.613971
vid11                209.008402  109.885864
vid12                227.466912  125.311414
vid13                218.641919  111.928529
vid14                208.970018  122.657554
vid15                164.374772  146.652589
vid2                 227.548949  120.021222
vid3                 212.584627  143.306575
vid4                 228.137296  117.314867
vid5                 212.086079  109.802457
vid6                 210.491175  123.785899
vid7                 215.913768  196.922698
vid8                 210.568375  115.228228
vid9                 189.571147  122.331552

In [8]:
def summary_statistics(df, vergence_angles, vergence_distance):
    df["Veregence Angles [rad]"] = vergence_angles
    df["Pupil Distance [px]"] = vergence_distance
    df_groups_sti = df.groupby("Stimulus") 
    statistics = ["mean", "std"]

    df_summary_statistics = pd.DataFrame(index = ["vid1","vid10","vid11","vid12","vid13","vid14","vid15","vid2","vid3","vid4","vid5","vid6","vid7","vid8","vid9"])
    df_summary_statistics["Participant"] = [df.loc[:,"Participant"].values[0]] * 15

    vergence = df_groups_sti.agg({"Veregence Angles [rad]": statistics})
    vergence.columns = ["vergence_mean", "vergence_std"]
    df_summary_statistics[["Veregence Angles Mean [rad]", "Veregence Angles Std [rad]"]] = vergence

    distance = df_groups_sti.agg({"Pupil Distance [px]": statistics})
    distance.columns = ["distance_mean", "distance_std"]
    df_summary_statistics[["Pupil Distance Mean [px]","Pupil Distance Std [px]"]] = distance
    return df_summary_statistics 

In [39]:
sum  = summary_statistics(df, vergence_angles(df), vergence_distance(df))
sum
# summary statistics läuft und wirft keine Fehler

C:\Users\Julia\AppData\Local\Temp\ipykernel_31216\1287561581.py:3: RuntimeWarning: invalid value encountered in divide
  return vector / np.linalg.norm(vector)


,Participant,Veregence Angles Mean [rad],Veregence Angles Std [rad],Pupil Distance Mean [px],Pupil Distance Std [px]
vid1,1,0.024544,0.008193,219.578560,126.804463
vid10,1,0.016063,0.008325,220.563184,106.613971
vid11,1,0.033338,0.023195,209.008402,109.885864
vid12,1,0.030035,0.018617,227.466912,125.311414
vid13,1,0.024829,0.006304,218.641919,111.928529
vid14,1,0.022077,0.007194,208.970018,122.657554
vid15,1,0.026315,0.013422,164.374772,146.652589
vid2,1,0.027963,0.011599,227.548949,120.021222
vid3,1,0.027969,0.013124,212.584627,143.306575
vid4,1,0.021957,0.013046,228.137296,117.314867


In [9]:
def all_angles():
    os.chdir("C:/Users/Julia/Desktop/Daten-BA/SMI_RawData")
    #get all files in directory
    list = os.listdir() 
    
    # remove participants
    #list.remove("Event Statistics - Single - 82.txt")
    vergence = pd.DataFrame()
    summary_statistics_all_participants = pd.DataFrame()
    df_summary_statistic_all_participants = pd.DataFrame()
    for filename in list:
        print(filename)
        # input: filename # output: dataframe to work on
        df = read_data(filename)
        
        df_10_sec_of_videos = cut_last_10sec_of_videos(df)
        
        list_vergence_angles = vergence_angles(df_10_sec_of_videos)
        serie_vergence_distance = vergence_distance(df)
        
        df_summary_statistics = summary_statistics(df_10_sec_of_videos, list_vergence_angles, serie_vergence_distance)
        df_summary_statistic_all_participants = pd.concat([df_summary_statistic_all_participants, df_summary_statistics])

    return df_summary_statistic_all_participants


In [10]:
all = all_angles()

Raw Data - Raw Data - 01.txt


C:\Users\Julia\AppData\Local\Temp\ipykernel_8928\3496983591.py:9: DtypeWarning: Columns (46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(
C:\Users\Julia\AppData\Local\Temp\ipykernel_8928\1287561581.py:3: RuntimeWarning: invalid value encountered in divide
  return vector / np.linalg.norm(vector)


Raw Data - Raw Data - 02.txt


C:\Users\Julia\AppData\Local\Temp\ipykernel_8928\3496983591.py:9: DtypeWarning: Columns (46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(
C:\Users\Julia\AppData\Local\Temp\ipykernel_8928\1287561581.py:3: RuntimeWarning: invalid value encountered in divide
  return vector / np.linalg.norm(vector)


Raw Data - Raw Data - 03.txt


C:\Users\Julia\AppData\Local\Temp\ipykernel_8928\3496983591.py:9: DtypeWarning: Columns (46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(
C:\Users\Julia\AppData\Local\Temp\ipykernel_8928\1287561581.py:3: RuntimeWarning: invalid value encountered in divide
  return vector / np.linalg.norm(vector)


Raw Data - Raw Data - 04.txt


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [42]:
all

,Participant,Veregence Angles Mean [rad],Veregence Angles Std [rad],Pupil Distance Mean [px],Pupil Distance Std [px]
vid1,1,0.024544,0.008193,219.578560,126.804463
vid10,1,0.016063,0.008325,220.563184,106.613971
vid11,1,0.033338,0.023195,209.008402,109.885864
vid12,1,0.030035,0.018617,227.466912,125.311414
vid13,1,0.024829,0.006304,218.641919,111.928529
...,...,...,...,...,...
vid5,37,0.065063,0.017223,319.210456,175.464820
vid6,37,0.077040,0.048133,474.755150,347.142264
vid7,37,0.059891,0.020194,298.836828,186.256348
vid8,37,0.048726,0.019520,273.396230,150.697973


In [20]:
all["Participant"]

vid1      1
vid10     1
vid11     1
vid12     1
vid13     1
         ..
vid5     37
vid6     37
vid7     37
vid8     37
vid9     37
Name: Participant, Length: 555, dtype: object

In [ ]:
## vergence for one participant
# möchte es für alle participants
# und möchte es in meiner summary statitsic dabei haben
# kann es in meiner main funktion aufrufen, mache alles in extra file fertig, sodass ein funktionsaufruf reicht um es hinzuzufügen

